# Using SNOWSLIDE in the OGGM Workflow

Ce notebook a pour objectif de guider l'utilisateur à travers l'utilisation de snowslide combiné au workflow OGGM. Il nécessite le téléchargement du package snowslide : 'https://github.com/GuillemCF/Snowslide.git'. Les informations relatives à l'utilisation de ce package sont disponibles dans le readme. Il nécessite également l'installation du modèle oggm dont les détails d'installation peuvent être retrouvés à l'adresse suivante : 'https://docs.oggm.org/en/stable/installing-oggm.html'.
Une fois installés, l'utilisateur devra se plaver dans un environnement où snowslide et oggm sont évidemment installés (ex oggm_env comme expliqué dans le tutoriel d'installation d'OGGM)

In [8]:
# Importation des librairies nécessaires à OGGM
from oggm import cfg
from oggm import tasks, utils, workflow, graphics
from oggm.core import massbalance
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

In [9]:
import warnings
# Some annoying warnings sometimes
warnings.filterwarnings(action='ignore', category=UserWarning)

## Pick a glacier 
On initialise le workflow et on créé un dossier qui contiendra tous les directories propres au fonctionnement d'OGGM. Modifier dir_path par le chemin d'accès au dossier où on veut que les données soient écrites. 

In [10]:
# Initialize OGGM and set up the default run parameters
cfg.initialize(logging_level='INFO')
dir_path = utils.get_temp_dir('snowslide')
# Local working directory (where OGGM will write its output)
cfg.PATHS['working_dir'] = utils.mkdir(dir_path)

2023-11-09 11:51:21: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2023-11-09 11:51:21: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2023-11-09 11:51:21: oggm.cfg: Multiprocessing: using all available processors (N=8)


On choisit les glaciers sur lesquels on veut travailler et on rendre leur index RGI dans une liste rgi_ids. Dans cet exemple on ne le fait que sur un seul glacier (mais on peut rentrer une liste de rgi_ids)

In [11]:
# rgi_ids = ['RGI60-11.01450']  # This is Aletsch
# rgi_ids = ['RGI60-11.00897']  # This is Hintereisferner
rgi_ids = ['RGI60-11.03466','RGI60-11.00897']  # This is Talefre 

# This is the url with loads of data (dhdt, velocities, etc)
base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.1/elev_bands/W5E5_w_data/'

# Can be replaced with 
# https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.1/elev_bands/W5E5 
# for smaller data

# This gets the data for this glacier - download can be a bit long because of all the data
gdirs = workflow.init_glacier_directories(rgi_ids, prepro_base_url=base_url, from_prepro_level=3, prepro_border=80)

2023-11-09 11:51:21: oggm.workflow: init_glacier_directories from prepro level 3 on 2 glaciers.
2023-11-09 11:51:21: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 2 glaciers


A ce stade les glacier_directories ont été créés et les données de base y sont présentes (voir dans le dossier indiqué avec dir_path). La task incluant snowslide dans OGGM a donc pour objectif de récupérer ces données, lancer une simulation snowslide et réécrire les sorties dans les glacier_directories. Aussi simple que ça !

## A task with OGGM 

On importe snowslide sous forme de task OGGM (foction associée au package python et intitulée 'oggm_snowslide_compat)


In [12]:
from snowslide import oggm_snowslide_compat

In [13]:
# run the tasks
workflow.execute_entity_task(oggm_snowslide_compat.snowslide_to_gdir, gdirs)

2023-11-09 11:51:22: oggm.workflow: Execute entity tasks [snowslide_to_gdir] on 2 glaciers
2023-11-09 11:51:22: snowslide.oggm_snowslide_compat: (RGI60-11.03466) snowslide_to_gdir
2023-11-09 11:51:24: snowslide.snowslide_main: (RGI60-11.03466) the algorithm converged in 60 iterations
2023-11-09 11:51:24: snowslide.oggm_snowslide_compat: (RGI60-11.00897) snowslide_to_gdir
2023-11-09 11:51:25: snowslide.snowslide_main: (RGI60-11.00897) the algorithm converged in 34 iterations


[None, None]

La task a été lancée et snowslide a tourné, les résultats sont enregistré dans le fichier gridded_data sous la forme 'snowslide_1m_avalanches'. On a également créé une fonction qui calcule des statistiques pour les glaciers sélectionnés avec notamment : le dépôt moyen due aux avalanches sur le glacier, l'aire de dépôt et le volume total de dépôt dues aux avalanches. Il faut noter que pour l'instant avec cette task l'utilisateur n'a pas le choix dans l'initialisation des hauteurs de neige qui se fait avec 1m de neige partout sur le dem. (il sera très facile de laisser le choix à l'utilisateur d'implémenter avec des données réelles par la suite...). Les statistiques sont accessibles avec les commandes suivantes : 

In [14]:
odf = oggm_snowslide_compat.compile_snowslide_statistics(gdirs)
odf

2023-11-09 11:51:25: snowslide.oggm_snowslide_compat: Applying global task compile_snowslide_statistics on 2 glaciers
2023-11-09 11:51:25: oggm.workflow: Execute entity tasks [snowslide_statistics] on 2 glaciers
2023-11-09 11:51:25: snowslide.oggm_snowslide_compat: (RGI60-11.03466) snowslide_statistics
2023-11-09 11:51:25: snowslide.oggm_snowslide_compat: (RGI60-11.00897) snowslide_statistics


,rgi_region,rgi_subregion,rgi_area_km2,map_dx,snowslide_1m_glacier_average,snowslide_deposit_area_km2,snowslide_deposit_volume_km3
rgi_id,,,,,,,
RGI60-11.03466,11,11-01,7.687,49.0,1.1881187,1.418991,0.003154
RGI60-11.00897,11,11-01,8.036,50.0,1.014873,0.545000,0.000780
